In [ ]:
from 

In [ ]:
import json
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
from IPython.display import display
from PIL import Image

pd.set_option("display.max_colwidth", None)  # Ensures full text is shown
pd.set_option("display.max_rows", 200)  # Increase max rows if needed
pd.set_option("display.max_columns", 50)  # Increase max columns if needed

In [ ]:
ann_path = "/data/SSD/flickr30k/annotations/test.json"
img_path = "/data/SSD/flickr30k/images/"

ann = json.load(open(ann_path, "r"))
txt_collection = [item["caption"] for item in ann]
txt_collection = [item for sublist in txt_collection for item in sublist]

eval_results = torch.load("./eval_results_top_110_flickr30k.pt")

print(eval_results.keys())

sorted_indices_label_collection = eval_results[
    "sorted_indices_label_collection"
]  # list of length N (number of labels)
all_selected_label_indices_collection = eval_results[
    "all_selected_label_indices_collection"
]  # nested list of length N, where each item is the index which the image choose this label
inds_collection = eval_results["inds_collection"]  # nested list of length N
inds_raw_itt = eval_results["inds_raw_itt"]  # [1000, 5000]

In [ ]:
all_selected_label_indices_collection[10]

In [ ]:
n = 0
n = min(n, len(sorted_indices_label_collection))
selected_indices = sorted_indices_label_collection[n]
print(int(selected_indices))

In [ ]:
i = 4

idx = all_selected_label_indices_collection[n][i]
check_top_k = 50
item = ann[idx]
img = os.path.join(img_path, item["image"])
img = Image.open(img).convert("RGB")
# turn off axis
plt.axis("off")
plt.imshow(img)
plt.show()

# First get original caption
original_caption = item["caption"]

# Then get the caption retrieved by raw
retrived_caption_index_raw = inds_raw_itt[idx][:check_top_k]
retrived_caption_raw = [txt_collection[i] for i in retrived_caption_index_raw]

# Finally get the caption retrieved by our method
retrived_caption_index_cdc = inds_collection[n][idx][:check_top_k].tolist()
retrived_caption_cdc = [txt_collection[i] for i in retrived_caption_index_cdc]

# Turn into a panda dataframe
df = pd.DataFrame(
    {
        "Raw_retrieve": retrived_caption_raw,
        "CDC_retrieve": retrived_caption_cdc,
    }
)


# Function to highlight duplicates
def highlight_duplicates(val, col1, col2):
    # If the value appears in both columns, color it blue

    if val in original_caption:
        return "background-color: lightgreen"

    if val in df[col1].values and val in df[col2].values:
        return "background-color: lightblue"

    return ""


# Display the dataframe
with pd.option_context("display.max_colwidth", None):
    styled_df = df.style.map(highlight_duplicates, col1="Raw_retrieve", col2="CDC_retrieve")
    display(styled_df)